In [34]:
!wget http://www.eraserbenchmark.com/zipped/multirc.tar.gz
!tar -xvzf multirc.tar.gz
!mv multirc/val.jsonl multirc/dev.jsonl

In [37]:
import json
from rationale_benchmark.utils import load_flattened_documents
documents = load_flattened_documents('multirc', docids=None)
def read_annotations(json_file) :
    anns = [json.loads(line) for line in open(json_file)]
    for a in anns :
        doc_id = a['annotation_id'].split(':')
        assert len(doc_id) == 3
        doc_id = doc_id[0]
        a['document'] = " ".join(documents[doc_id])
        a['label'] = a['classification']
        del a['classification']
        del a['docids']
        a['rationale'] = []
        for evgroup in a['evidences'] :
            for ev in evgroup :
                assert ev['docid'] == doc_id
                a['rationale'].append((ev['start_token'], ev['end_token']))
        del a['evidences']
        del a['query_type']
        
    return anns

In [38]:
import os
os.makedirs('data/', exist_ok=True)

In [39]:
for key in ['train', 'dev', 'test'] :
    ann = read_annotations('multirc/' + key + '.jsonl')
    with open('data/' + key + '.jsonl', 'w') as f :
        f.write('\n'.join([json.dumps(line) for line in ann]))